In [1]:
!pip install -q --upgrade google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 1.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import getpass
import os
import google.generativeai as genai

# 실행하기 전에 열쇠창에 API키를 등록하세요.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

model = genai.GenerativeModel('gemini-pro')

In [5]:
# 종종 실행이 안됩니다. 그럴경우 런타임 연결해제 및 재시작 후 다시 시도하세요.
import time
import re
from tqdm.notebook import tqdm
from google.generativeai import types as genai

# 소설의 시작 문장을 작성해 주세요.
prompt1 = "새싹에서 주관하는 구글클라우드인공지능개발자 과정을 수강중인 소프트웨어 개발자를 준비하는 청년들이 있었다. "

# 생성 규칙을 지정해 주세요.
prompt2 = """{당신은 한국의 매우 뛰어난 실력을 갖춘 저명한 소설 작가입니다. 이어지는 소설을 길게 작성해주세요.
스토리가 아주아주 조금씩 전개되도록 작성해주세요. 아래의 모든 조건들을 반드시 반드시 반드시!! 지켜주세요.
조건1 : 영어 사용금지.
조건2 : 중국어 사용금지.
조건3 : 숫자 사용금지.
조건4 : 스토리가 이어질 수 있는 문장으로 마무리 하시오.
조건5 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건6 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건7 : 스토리의 결말을 지어서는 안됨.
조건8 : 줄바꿈을 적절히 활용하시오.
조건9 : 대명사 사용을 자제하고, 주인공의 이름을 지속적으로 사용하시오.
조건10 : 주인공과 친구들은 절대 사망해서는 안됩니다.
조건11 : 인물들간에 대화를 자주 삽입 하시오.
조건12 : 언어는 한국어 이외에 절대 사용 금지.
조건13 : 비슷한 문장 반복 자제할것. 동일 단어 반복 자제할것.
조건14 : 주인공은 완규입니다.
조건15 : 동료는 규미, 지호, 창석, 시내입니다.}"""

# 마무리 규칙을 지정해 주세요.
prompt3 = """{이어지는 내용으로 소설작성을 마무리 하시오. 또한 아래의 조건을 준수하시오.
조건1 : 진행되고 있던 소설을 매끄럽게 마무리 합니다. 확실하게 결말을 지으세요. 마무리 이후에는 더 이상의 아무런 문장도 생성하지 않습니다.
조건2 : 영어 사용금지.
조건3 : 중국어 사용금지.
조건4 : 숫자 사용금지.
조건5 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건6 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건7 : 줄바꿈을 적절히 활용하시오.
조건8 : 언어는 한국어 이외에 사용 금지.}"""

prompt1_copy = prompt1


# 생성횟수를 설정하세요.
n = 25
myNovel = ""
current_prompt = ""
total_tokens_accumulated = 0

def count_tokens(model, prompt):
    try:
        response = model.count_tokens(prompt)
        total_tokens = response.total_tokens
        return total_tokens
    except Exception as e:
        print(f"토큰 개수 확인 중 오류 발생: {e}")
        return None

def generate_selected_sentences(text):
    sentences = text.split('.')
    selected_sentences = ""
    intervals = [(i * (i + 1)) // 2 for i in range(1, 35)]
    reversed_intervals = intervals[::-1]
    for index in reversed_intervals:
        try:
            selected_sentences = '.'.join(sentences[-index:]).strip()
            break
        except IndexError as ie:
            print(f"IndexError 발생: {ie}")
            continue
    return selected_sentences

for i in tqdm(range(n), desc="Generating Text"):
    print()
    print(f"*** 현재 {i+1}번째 반복 중 ***")
    while True:
        current_prompt = prompt1 + prompt3 if i == n-1 else prompt1 + prompt2
        generation_config = genai.GenerationConfig(temperature=1.0)
        response = model.generate_content(current_prompt, generation_config=generation_config)
        input_tokens = count_tokens(model, current_prompt)
        print(f"사용된 입력 토큰: {input_tokens}")
        total_tokens = count_tokens(model, response.text)
        print(f"사용된 출력 토큰: {total_tokens}")
        total_tokens_accumulated += total_tokens
        try:
            if response.candidates and response.parts:
                generated_text = response.parts[0].text
                consecutive_errors = 0
            else:
                print("모델이 적절한 후보를 생성하지 못했습니다. 재시도합니다.")
                consecutive_errors += 1
                continue
            if re.search("[^가-힣\s\n!\"'\.,?~()]", generated_text):
                print(f"생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.")
                continue
        except ValueError as ve:
            print(f"ValueError 발생: {ve}")
            consecutive_errors += 1
            generated_text = response.parts[0].text
        except IndexError as ie:
            print(f"IndexError 발생: {ie}")
            consecutive_errors += 1
            continue
        if consecutive_errors >= 3:
            print("연속된 예외 발생으로 인해 종료합니다.")
            break
        selected_sentences = generate_selected_sentences(myNovel)
        prompt1 = selected_sentences + '\n'
        myNovel += generated_text
        break
    print(f"현재까지 사용된 출력 토큰의 누적 개수: {total_tokens_accumulated}")

myNovel = prompt1_copy + myNovel
print(myNovel)

Generating Text:   0%|          | 0/25 [00:00<?, ?it/s]


*** 현재 1번째 반복 중 ***
사용된 입력 토큰: 420
사용된 출력 토큰: 960
현재까지 사용된 출력 토큰의 누적 개수: 960

*** 현재 2번째 반복 중 ***
사용된 입력 토큰: 378
사용된 출력 토큰: 834
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 378
사용된 출력 토큰: 623
현재까지 사용된 출력 토큰의 누적 개수: 2417

*** 현재 3번째 반복 중 ***
사용된 입력 토큰: 1338
사용된 출력 토큰: 683
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 1338
사용된 출력 토큰: 685
현재까지 사용된 출력 토큰의 누적 개수: 3785

*** 현재 4번째 반복 중 ***
사용된 입력 토큰: 1961
사용된 출력 토큰: 1232
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 1961
사용된 출력 토큰: 906
현재까지 사용된 출력 토큰의 누적 개수: 5923

*** 현재 5번째 반복 중 ***
사용된 입력 토큰: 2646
사용된 출력 토큰: 892
현재까지 사용된 출력 토큰의 누적 개수: 6815

*** 현재 6번째 반복 중 ***
사용된 입력 토큰: 3551
사용된 출력 토큰: 1576
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 3551
사용된 출력 토큰: 586
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 3551
사용된 출력 토큰: 790
현재까지 사용된 출력 토큰의 누적 개수: 9767

*** 현재 7번째 반복 중 ***
사용된 입력 토큰: 4443
사용된 출력 토큰: 622
현재까지 사용된 출력 토큰의 누적 개수: 10389

*** 현재 8번째 반복 중 ***
사용된 입력 토큰: 5233
사용된 출력 토큰: 753
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.
사용된 입력 토큰: 5233
사용된 출력 토큰: 845
생성된 텍스트에 노이즈가 있습니다. 재실행 합니다.


In [ ]:
# 파일을 저장합니다. 드라이브 마운트를 해주세요.
from datetime import datetime
import os

directory_path = "/content/drive/MyDrive/novel/"

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(directory_path, f"{current_time}_novel.txt")
combined_prompt_path = os.path.join(directory_path, f"{current_time}_prompt.txt")

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

with open(file_path, "w", encoding="utf-8") as file:
    file.write(myNovel)
print(f"Novel 저장완료 : {file_path}")

with open(combined_prompt_path, "w", encoding="utf-8") as prompt_file:
    prompt_file.write(f"Sum of Token: {total_tokens_accumulated}\n")
    prompt_file.write("\nPrompt 1:\n")
    prompt_file.write(prompt1_copy + "\n")
    prompt_file.write("\nPrompt 2:\n")
    prompt_file.write(prompt2 + "\n")
    prompt_file.write("\nPrompt 3:\n")
    prompt_file.write(prompt3)
print(f"Prompt 저장완료 : {combined_prompt_path}")